In [1]:
!pip install category_encoders
!pip install wandb
!pip install onnx
!pip install skl2onnx
!pip install onnxruntime
!pip install onnxmltools

     |████████████████████████████████| 82 kB 345 kB/s 
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 180 kB 51.9 MB/s 
     |████████████████████████████████| 140 kB 57.3 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=4234f64a4a7831bbb762fda46c1b9764afb5b360242f59292f615d7641e46581
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=01e469f854f9e375d0964490e86b529280782a613a6229e864363faef81e0d88
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 12.7 MB 5.3 MB/s 
     |███████████████████████

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
#import wandb
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer
import onnxruntime as rt
from skl2onnx import convert_sklearn, __version__, update_registered_converter
from skl2onnx.common.data_types import FloatTensorType, StringTensorType
from skl2onnx.common.data_types import Int64TensorType
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  # noqa
import pprint
import os
import wandb
from xgboost import XGBClassifier
#from gradio import gradio as gr

warnings.filterwarnings("ignore")

In [56]:
from imblearn.over_sampling import RandomOverSampler


In [24]:
from sklearn.metrics import confusion_matrix

In [3]:
os.environ["WANDB_API_KEY"] = "210e9521dcd2f2916b20d49fc033bcd05cf4ac61"

# Load Dataset

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/rajap20/streamlit-example/master/data_final.csv')

In [5]:
df.head(5)

,Customer_id,Date_disb,pay_type,area_code,pin_code,state,dealer,product_code,tenure,roi,emi,proc_fee,asset_cost,loan_amt,gender,qualification,employ_type,resid_type,age,cibil_score,net_salary,net_irr,fraud
0,0,2/13/2020,ACH,3075,530041,AP,DEALER,SC,0.470588,-0.793282,0.124252,-0.093758,87000,71000,M,OTHERS,SAL,O,0.306122,-0.906781,0.158169,-1.321153,0
1,1,06-02-2020,ADM,3075,532123,AP,DEALER,SC,0.352941,-0.952045,0.083863,-0.572295,61500,51000,F,PG,SAL,O,0.551020,0.163204,1.161336,-1.308269,0
2,2,10-02-2020,ADM,3075,533218,AP,ASC,SC,0.352941,0.318061,0.210079,0.913555,87000,74038,M,SSC,SEP,O,0.530612,NaN,0.910545,0.605033,0
3,3,10-02-2020,ADM,3075,532484,AP,DEALER,SC,0.352941,-1.309262,0.166511,0.649593,84000,69720,M,OTHERS,AGR,O,0.224490,-0.167982,-0.544048,-1.098901,0
4,4,10-02-2020,ADM,3075,530046,AP,DEALER,SC,0.352941,-0.713900,0.092091,-0.572295,61500,52220,F,HSC,SAL,O,0.224490,NaN,-0.343415,-0.986164,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11702 entries, 0 to 11701
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Customer_id    11702 non-null  int64  
 1   Date_disb      11702 non-null  object 
 2   pay_type       11702 non-null  object 
 3   area_code      11702 non-null  int64  
 4   pin_code       11702 non-null  int64  
 5   state          11702 non-null  object 
 6   dealer         11702 non-null  object 
 7   product_code   11702 non-null  object 
 8   tenure         11702 non-null  float64
 9   roi            11702 non-null  float64
 10  emi            11702 non-null  float64
 11  proc_fee       11702 non-null  float64
 12  asset_cost     11702 non-null  int64  
 13  loan_amt       11702 non-null  int64  
 14  gender         11698 non-null  object 
 15  qualification  11698 non-null  object 
 16  employ_type    11698 non-null  object 
 17  resid_type     11698 non-null  object 
 18  age   

# Feature Set Selection




In [7]:
df.columns

Index(['Customer_id', 'Date_disb', 'pay_type', 'area_code', 'pin_code',
       'state', 'dealer', 'product_code', 'tenure', 'roi', 'emi', 'proc_fee',
       'asset_cost', 'loan_amt', 'gender', 'qualification', 'employ_type',
       'resid_type', 'age', 'cibil_score', 'net_salary', 'net_irr', 'fraud'],
      dtype='object')

In [8]:
x_features = [
              'area_code', 
              'state', 'resid_type', 'net_irr', 'proc_fee',
       'asset_cost', 'loan_amt', 'emi', 'net_salary', 'roi', 'tenure',
       'age']

In [9]:
cat_features = [
                'area_code', 
                'state', 'resid_type']

In [10]:
num_features = list(set(x_features) - set(cat_features))
num_features

['emi',
 'tenure',
 'age',
 'loan_amt',
 'roi',
 'proc_fee',
 'asset_cost',
 'net_salary',
 'net_irr']

In [11]:
df[x_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11702 entries, 0 to 11701
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_code   11702 non-null  int64  
 1   state       11702 non-null  object 
 2   resid_type  11698 non-null  object 
 3   net_irr     11702 non-null  float64
 4   proc_fee    11702 non-null  float64
 5   asset_cost  11702 non-null  int64  
 6   loan_amt    11702 non-null  int64  
 7   emi         11702 non-null  float64
 8   net_salary  11702 non-null  float64
 9   roi         11702 non-null  float64
 10  tenure      11702 non-null  float64
 11  age         11698 non-null  float64
dtypes: float64(7), int64(3), object(2)
memory usage: 1.1+ MB


In [12]:
df[cat_features] = df[cat_features].astype(object)
df[x_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11702 entries, 0 to 11701
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_code   11702 non-null  object 
 1   state       11702 non-null  object 
 2   resid_type  11698 non-null  object 
 3   net_irr     11702 non-null  float64
 4   proc_fee    11702 non-null  float64
 5   asset_cost  11702 non-null  int64  
 6   loan_amt    11702 non-null  int64  
 7   emi         11702 non-null  float64
 8   net_salary  11702 non-null  float64
 9   roi         11702 non-null  float64
 10  tenure      11702 non-null  float64
 11  age         11698 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 1.1+ MB


In [13]:
df = df[x_features + ['fraud']].dropna()
df.shape

(11698, 13)

In [14]:
X = df[x_features]
y = df['fraud']

In [15]:
cat_index = [0, 1, 2]

In [16]:
"""X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size = 0.8,
                                                    random_state = 80)"""

'X_train, X_test, y_train, y_test = train_test_split(X,\n                                                    y,\n                                                    train_size = 0.8,\n                                                    random_state = 80)'

# Function Defining

In [59]:
def prepare_data(X,
                 y,
#                 num_scaler = None,
#                 num_imputer = None,
#                 cat_imputer = None,
                 cat_encoder,
                 num_scaler,
                 num_features,
                 cat_features,
                 train_size = 0.8,
                 seed = 100):
  
  # Split the dataset into train and test split
  x_train, x_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      train_size = train_size,
                                                      random_state = seed)
  
  
  # encoder the categorical features
  cat_encoder.fit(X=x_train, y=y_train)
  x_train_encoded = cat_encoder.transform(x_train)
  x_test_encoded = cat_encoder.transform(x_test)

  # scaling numeric features
  num_scaler.fit(x_train[num_features])
  x_train_scaler = num_scaler.transform(x_train[num_features])
  x_test_scaler = num_scaler.transform(x_test[num_features])

  return x_train_encoded, x_test_encoded, y_train, y_test



def prepare_data_SMOTE(X,
                 y,
#                 num_imputer = None,
#                 cat_imputer = None,
                 cat_encoder,
                 num_scaler,
                 num_features,
                 cat_features,
                 train_size = 0.8,
                 seed = 100):
  
  # Split the dataset into train and test split
  x_train, x_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      train_size = train_size,
                                                      random_state = seed)
  
  oversample = SMOTENC(categorical_features=cat_features, random_state=seed)
  x_train_sm, y_train_sm = oversample.fit_resample(x_train, y_train) 
  
  # encoder the categorical features
  cat_encoder.fit(X=x_train_sm, y=y_train_sm)
  x_train_encoded = cat_encoder.transform(x_train_sm)
  x_test_encoded = cat_encoder.transform(x_test)

  """oversample = SMOTE()
  x_train_sm, y_train_sm = oversample.fit_resample(x_train_encoded, y_train)

  return x_train_sm, x_test_encoded, y_train_sm, y_test"""

  return x_train_encoded, x_test_encoded, y_train_sm, y_test


def prepare_data_OS(X,
                 y,
#                 num_imputer = None,
#                 cat_imputer = None,
                 cat_encoder,
                 num_scaler,
                 num_features,
                 cat_features,
                 train_size = 0.8,
                 seed = 100):
  
  # Split the dataset into train and test split
  x_train, x_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      train_size = train_size,
                                                      random_state = seed)
  
  ros = RandomOverSampler(random_state=0)
  x_train_sm, y_train_sm = ros.fit_resample(x_train, y_train)

  
  # encoder the categorical features
  cat_encoder.fit(X=x_train_sm, y=y_train_sm)
  x_train_encoded = cat_encoder.transform(x_train_sm)
  x_test_encoded = cat_encoder.transform(x_test)

  """oversample = SMOTE()
  x_train_sm, y_train_sm = oversample.fit_resample(x_train_encoded, y_train)

  return x_train_sm, x_test_encoded, y_train_sm, y_test"""

  return x_train_encoded, x_test_encoded, y_train_sm, y_test
  

In [18]:
def get_total_cost( actual, predicted, cost_TNs, cost_FNs ):
    # Get the confusion matrix and calculate cost
    tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
    return tn * cost_TNs + fn * cost_FNs

def run_model_experiment(model,
                         x_train,
                         y_train,
                         x_test,
                         y_test,
                         labels,
                         name,
                         config = None,
                         tags = None):
  
  
  model.fit(x_train, y_train)

  wandb.init(project='mlops', config=config, tags = tags)
  wandb.run.name = name

  test_df=pd.DataFrame({
                      "y_test":y_test,
                       "y_prob":model.predict_proba(x_test)[:,1]
                      })
  cost_df = pd.DataFrame( columns = ['prob', 'cost'])
  idx = 0
  cost_FPs = 350
  cost_TPs = -84

  ## Iterate cut-off probability values between 0.1 and 0.5
  for each_prob in range(0,100):
    cost = get_total_cost(test_df.y_test, test_df.y_prob.map(lambda x: 1 if x > (each_prob/100) else 0), cost_TPs, cost_FPs)
    cost_df.loc[idx] = [(each_prob/100), cost]
    idx += 1

  cost_df = cost_df.sort_values( 'cost', ascending = True )
  min_cost = cost_df.iloc[0,1]
  cutoff_prob = cost_df.iloc[0,0]

  test_df['y_pred'] = test_df.y_prob.map(lambda x: 1 if x > cutoff_prob else 0)

  

  clasf_matrix = confusion_matrix(test_df.y_test, test_df.y_pred)
  sensitivity = [round(clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[1,0]),4)]
  specificity = [round(clasf_matrix[0,0]/(clasf_matrix[0,1]+clasf_matrix[0,0]),4)]
  precision = [round(clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[0,1]),4)]
  accuracy = [round((clasf_matrix[1,1]+clasf_matrix[0,0])/(clasf_matrix[1,1]+clasf_matrix[1,0]+clasf_matrix[0,1]+clasf_matrix[0,0]),4)]
  F_Score = [round(2*(clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[0,1]))*(clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[1,0]))/((clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[0,1])) + (clasf_matrix[1,1]/(clasf_matrix[1,1]+clasf_matrix[1,0]))), 4)]
  cost = min_cost

  print(clasf_matrix)

  wandb.log( {
      "sensitivity" : sensitivity, 
      "specificity": specificity,
      "precision" : precision,
      "accuracy" : accuracy,
      "F_Score" : F_Score,
      "cost" : cost})
  
  y_probas = test_df["y_prob"]
  
  
  """wandb.sklearn.plot_classifier(model, 
                               x_train, 
                               x_test, 
                               y_train, 
                               y_test,
                               y_probas,
                               lables,  
                               model_name=name,
                              feature_names=None)"""
  
  wandb.Artifact(name, 
                 type = 'model',
                 description = config)

  wandb.save()
  wandb.finish()

  return model, cutoff_prob

In [19]:
from category_encoders import OneHotEncoder, TargetEncoder


ohe_encoder = OneHotEncoder(cols = cat_features,
                            use_cat_names = True)

target_encoder = TargetEncoder(cols = cat_features)


std_scaler = StandardScaler()


# Baseline Model - Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

dtree_model = DecisionTreeClassifier(max_depth = 15)  

lables = [0,1]

dtree_model = run_model_experiment(dtree_model,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'DecisionTreeWithOHE',
                                   tags = ['Decision Tree', 'baseline', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2084   82]
 [ 167    7]]


cost,▁
cost,-116606.0


# Random Forest with OHE

In [30]:
x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 301,
           "max_depth": 20,
           "max_features": .6,
           "max_samples": 0.4 }

from sklearn.ensemble import RandomForestClassifier

rf_model_ohe1 = RandomForestClassifier(**params) 

rf_model_ohe1 = run_model_experiment(rf_model_ohe1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHE_HyPm1',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2130   36]
 [ 162   12]]


cost,▁
cost,-122220.0


In [31]:
params = { "n_estimators": 100,
           "max_depth": 10,
           "max_features": .6,
           "max_samples": 0.4 }

from sklearn.ensemble import RandomForestClassifier

rf_model_ohe2 = RandomForestClassifier(**params) 

rf_model_ohe2 = run_model_experiment(rf_model_ohe2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHE_HyPm2',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2153   13]
 [ 168    6]]


cost,▁
cost,-122052.0


In [32]:
params = { "n_estimators": 200,
           "max_depth": 15,
           "max_features": .6,
           "max_samples": 0.4 }

from sklearn.ensemble import RandomForestClassifier

rf_model_ohe3 = RandomForestClassifier(**params) 

rf_model_ohe3 = run_model_experiment(rf_model_ohe3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHE_HyPm3',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2124   42]
 [ 158   16]]


cost,▁
cost,-123116.0


# XGBoost with OHE

In [33]:
x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }



xgb_model_ohe1 = XGBClassifier(**params) 

xgb_model_ohe1, xgb_model_ohe_cutoff1 = run_model_experiment(xgb_model_ohe1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHE_HyPm1',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2042  124]
 [ 141   33]]


cost,▁
cost,-122178.0


In [34]:
params = { 
          "n_estimators": 20,
          "learning_rate": 0.05 ,
          "max_depth": 10,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }



xgb_model_ohe2 = XGBClassifier(**params) 

xgb_model_ohe2, xgb_model_ohe_cutoff2 = run_model_experiment(xgb_model_ohe2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHE_HyPm2',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2073   93]
 [ 148   26]]


cost,▁
cost,-122332.0


In [37]:
params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 30,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }



xgb_model_ohe3 = XGBClassifier(**params) 

xgb_model_ohe3, xgb_model_ohe_cutoff3 = run_model_experiment(xgb_model_ohe3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHE_HyPm3',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2149   17]
 [ 169    5]]


cost,▁
cost,-121366.0


# Random Forest with OHE with SMOTE

In [39]:
from imblearn.over_sampling import SMOTENC

In [40]:
x_train, x_test, y_train, y_test = prepare_data_SMOTE(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_index)

params = { "n_estimators": 301,
           "max_depth": 20,
           "max_features": .6,
           "max_samples": 0.4 }

rf_model_ohe_smote1 = RandomForestClassifier(**params) 

rf_model_ohe_smote1 = run_model_experiment(rf_model_ohe_smote1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithSMOTE_HyPm1',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2155   11]
 [ 167    7]]


cost,▁
cost,-122570.0


In [41]:
params = { "n_estimators": 200,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }

rf_model_ohe_smote2 = RandomForestClassifier(**params) 

rf_model_ohe_smote2 = run_model_experiment(rf_model_ohe_smote2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithSMOTE_HyPm2',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2147   19]
 [ 164   10]]


cost,▁
cost,-122948.0


In [42]:
params = { "n_estimators": 150,
           "max_depth": 10,
           "max_features": 0.6,
           "max_samples": 0.7 }

rf_model_ohe_smote3 = RandomForestClassifier(**params) 

rf_model_ohe_smote3 = run_model_experiment(rf_model_ohe_smote3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithSMOTE_HyPm3',
                                   tags = ['Random Forest', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2152   14]
 [ 168    6]]


cost,▁
cost,-121968.0


# XGBoost with OHE and SMOTE

In [43]:
x_train, x_test, y_train, y_test = prepare_data_SMOTE(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_index)

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

from xgboost import XGBClassifier


lables = [0,1]

xgb_model_ohe_smote1 = XGBClassifier(**params) 

xgb_model_ohe_smote1 = run_model_experiment(xgb_model_ohe_smote1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSMOTE_HyPm1',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2152   14]
 [ 167    7]]


cost,▁
cost,-122318.0


In [44]:
params = { "n_estimators": 200,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }

xgb_model_ohe_smote2 = XGBClassifier(**params) 

xgb_model_ohe_smote2 = run_model_experiment(xgb_model_ohe_smote2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSMOTE_HyPm2',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2158    8]
 [ 167    7]]


cost,▁
cost,-122822.0


In [49]:
params = { "n_estimators": 400,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }

xgb_model_ohe_smote3 = XGBClassifier(**params) 

xgb_model_ohe_smote3 = run_model_experiment(xgb_model_ohe_smote3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSMOTE_HyPm3',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2048  118]
 [ 143   31]]


cost,▁
cost,-121982.0


# Decision Tree with Target Encoding

In [46]:

target_encoder = TargetEncoder(cols = cat_features)


x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = target_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

dtree_model_te1 = DecisionTreeClassifier(max_depth = 15)  

dtree_model_te1 = run_model_experiment(dtree_model_te1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'DecisionTreeWithTarget_HyPm1',
                                   tags = ['Decision Tree', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2031  135]
 [ 141   33]]


cost,▁
cost,-121254.0


In [47]:
dtree_model_te2 = DecisionTreeClassifier(max_depth = 10)  

dtree_model_te2 = run_model_experiment(dtree_model_te2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'DecisionTreeWithTarget_HyPm2',
                                   tags = ['Decision Tree', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2099   67]
 [ 158   16]]


cost,▁
cost,-121016.0


In [48]:
dtree_model_te3 = DecisionTreeClassifier(max_depth = 20)  

dtree_model_te3 = run_model_experiment(dtree_model_te3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'DecisionTreeWithTarget_HyPm3',
                                   tags = ['Decision Tree', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[1999  167]
 [ 136   38]]


cost,▁
cost,-120316.0


# Random Forest with Target Encoding

In [50]:
x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = target_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 301,
           "max_depth": 20,
           "max_features": .6,
           "max_samples": 0.4 }


rf_model_te1 = RandomForestClassifier(**params) 

rf_model_te1 = run_model_experiment(rf_model_te1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithTarget_HyPm1',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2151   15]
 [ 167    7]]


cost,▁
cost,-122234.0


In [51]:
params = { "n_estimators": 200,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }


rf_model_te2 = RandomForestClassifier(**params) 

rf_model_te2 = run_model_experiment(rf_model_te2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithTarget_HyPm2',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2081   85]
 [ 152   22]]


cost,▁
cost,-121604.0


In [52]:
params = { "n_estimators": 150,
           "max_depth": 10,
           "max_features": 0.6,
           "max_samples": 0.7 }


rf_model_te3 = RandomForestClassifier(**params) 

rf_model_te3 = run_model_experiment(rf_model_te3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithTarget_HyPm3',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2097   69]
 [ 154   20]]


cost,▁
cost,-122248.0


# XGBoost With Target Encoding

In [53]:
x_train, x_test, y_train, y_test = prepare_data(X,
                                                y,
                                                cat_encoder = target_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }


xgb_model_te1 = XGBClassifier(**params) 

xgb_model_te1 = run_model_experiment(xgb_model_te1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithTarget_HyPm3',
                                   tags = ['XGBoost', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2161    5]
 [ 171    3]]


cost,▁
cost,-121674.0


In [54]:
params = { "n_estimators": 200,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }

xgb_model_te2 = XGBClassifier(**params) 

xgb_model_te2 = run_model_experiment(xgb_model_te2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithTarget_HyPm2',
                                   tags = ['XGBoost', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2165    1]
 [ 173    1]]


cost,▁
cost,-121310.0


In [55]:
params = { "n_estimators": 400,
           "max_depth": 25,
           "max_features": .7,
           "max_samples": 0.5 }

xgb_model_te3 = XGBClassifier(**params) 

xgb_model_te3 = run_model_experiment(xgb_model_te3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithTarget_HyPm3',
                                   tags = ['XGBoost', 'experiment', 'TargetEncoder'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2048  118]
 [ 143   31]]


cost,▁
cost,-121982.0


# Random Forest with OHE and Simple Upsampling

In [67]:
x_train, x_test, y_train, y_test = prepare_data_OS(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 301,
           "max_depth": 20,
           "max_features": .6,
           "max_samples": 0.4 }

rf_model_ohe_su1 = RandomForestClassifier(**params) 

rf_model_ohe_su1 = run_model_experiment(rf_model_ohe_su1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithUpsampling_HyPm1',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2127   39]
 [ 160   14]]


cost,▁
cost,-122668.0


In [68]:
params = { "n_estimators": 200,
           "max_depth": 10,
           "max_features": .6,
           "max_samples": 0.4 }

rf_model_ohe_su2 = RandomForestClassifier(**params) 

rf_model_ohe_su2 = run_model_experiment(rf_model_ohe_su2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithUpsampling_HyPm2',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2145   21]
 [ 164   10]]


cost,▁
cost,-122780.0


In [69]:
params = { "n_estimators": 200,
           "max_depth": 25,
           "max_features": .6,
           "max_samples": 0.4 }

rf_model_ohe_su3 = RandomForestClassifier(**params) 

rf_model_ohe_su3 = run_model_experiment(rf_model_ohe_su3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithOHEWithUpsampling_HyPm3',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2145   21]
 [ 164   10]]


cost,▁
cost,-122780.0


# XGBoost with OHE and Simple Upsampling

In [70]:
x_train, x_test, y_train, y_test = prepare_data_OS(X,
                                                y,
                                                cat_encoder = ohe_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_index)

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

xgb_model_ohe_su1 = XGBClassifier(**params) 

xgb_model_ohe_su1 = run_model_experiment(xgb_model_ohe_su1,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSampling_HyPm1',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2163    3]
 [ 173    1]]


cost,▁
cost,-121142.0


In [71]:
params = { 
          "n_estimators": 200,
          "learning_rate": 0.1 ,
          "max_depth": 15,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

xgb_model_ohe_su2 = XGBClassifier(**params) 

xgb_model_ohe_su2 = run_model_experiment(xgb_model_ohe_su2,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSampling_HyPm2',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2165    1]
 [ 173    1]]


cost,▁
cost,-121310.0


In [72]:
params = { 
          "n_estimators": 400,
          "learning_rate": 0.1 ,
          "max_depth": 24,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

xgb_model_ohe_su3 = XGBClassifier(**params) 

xgb_model_ohe_su1 = run_model_experiment(xgb_model_ohe_su3,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithOHEwithSampling_HyPm3',
                                   tags = ['XGBoost', 'experiment', 'OheEncoder', 'SMOTE'])

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


[[2164    2]
 [ 173    1]]


cost,▁
cost,-121226.0


# Random Forest with Target Encoding and SMOTE

In [ ]:
x_train, x_test, y_train, y_test, category_encoder = prepare_data_SMOTE(X,
                                                y,
                                                cat_encoder = target_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { "n_estimators": 301,
           "max_depth": 20,
           "max_features": .6,
           "max_samples": 0.4 }

rf_model = RandomForestClassifier(**params) 

rf_model = run_model_experiment(rf_model,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'RandomForestWithTargetWithSMOTE',
                                   tags = ['Random Forest', 'experiment', 'TargetEncoder', 'SMOTE'])

# XGBoost with target encoding with SMOTE

In [ ]:
x_train, x_test, y_train, y_test = prepare_data_SMOTE(X,
                                                y,
                                                cat_encoder = target_encoder,
                                                num_scaler = std_scaler,
                                                num_features = num_features,
                                                cat_features = cat_features)

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

from xgboost import XGBClassifier

xgb_model = XGBClassifier(**params) 

xgb_model = run_model_experiment(xgb_model,
                                   x_train,
                                   y_train,
                                   x_test,
                                   y_test,
                                   lables,
                                   name = 'XGBoostWithTargetwithSMOTE',
                                   tags = ['XGBoost', 'experiment', 'TargetEncoder', 'SMOTE'])

NameError: ignored

# Best Model is XGBoost with One Hot Encoding

# REGISTERING ON W&B

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    y,
                                                    train_size = 0.8,
                                                    random_state = 80)

## Converting the pipeline to ONNX format

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MyTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, xgb_model_ohe_smote):
        self.xgb_model_ohe_smote = xgb_model_ohe_smote

    def fit(self, a, b):
        return self

    def transform(self, a):
      return self.xgb_model_ohe_smote.predict(a)

In [ ]:
"""from sklearn.pipeline import make_pipeline


pipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'), MyTransformer(xgb_model_ohe_smote))

pipe.fit(X_train, y_train)
pipe.transform(X_train)"""

"from sklearn.pipeline import make_pipeline\n\n\npipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'), MyTransformer(xgb_model_ohe_smote))\n\npipe.fit(X_train, y_train)\npipe.transform(X_train)"

In [ ]:
"""categorical_transformer = Pipeline(steps=[('onehot', 
                                           OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, cat_features),])

reg = Pipeline(steps=[('preprocessor', preprocessor),
                          ('xgb', MyTransformer(xgb_model_ohe_smote))
                          ])
reg.fit(X_train, 
        y_train)

reg.transform(X_test)"""

"categorical_transformer = Pipeline(steps=[('onehot', \n                                           OneHotEncoder(handle_unknown='ignore'))])\n\npreprocessor = ColumnTransformer(\n    transformers=[('cat', categorical_transformer, cat_features),])\n\nreg = Pipeline(steps=[('preprocessor', preprocessor),\n                          ('xgb', MyTransformer(xgb_model_ohe_smote))\n                          ])\nreg.fit(X_train, \n        y_train)\n\nreg.transform(X_test)"

In [28]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', 
                                           OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),                  
        ('cat', categorical_transformer, cat_features),
    ])

params = { 
          "n_estimators": 301,
          "learning_rate": 0.1 ,
          "max_depth": 20,
          "min_child_weight": 7,
          "gamma": 0.1,
          "colsample_bytree" : 0.7
          }

XGB = XGBClassifier(**params) 

reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', XGB)])           

reg.fit(X_train, 
        Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['emi', 'tenure', 'age',
                                                   'loan_amt', 'roi',
                                                   'proc_fee', 'asset_cost',
                                                   'net_salary', 'net_irr']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['area_code', 'state',
                                                   'resid_type'])])),
                ('classifier',
                 XGBClassif

In [29]:
import pickle 
pickle_out = open("classifier.pkl", mode = "wb") 
pickle.dump(reg, pickle_out) 
pickle_out.close()

In [ ]:
temp1 = pd.DataFrame({
                    "y_test":y_test,
                      "y_prob":reg.predict_proba(X_test)[:,1]
                    })

sum(temp1.y_prob.map(lambda x: 1 if x > xgb_model_ohe_cutoff else 0))

NameError: ignored

## Creating the ONNX Graph

In [ ]:
def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = FloatTensorType([None, 1])
        elif v == 'float64':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs


inputs = convert_dataframe_schema(X_train)

pprint.pprint(inputs)

- Registers or updates a converter for a new model so that it can be converted when inserted in a scikit-learn pipeline.

In [ ]:
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

In [ ]:
model_onnx = convert_sklearn(
    reg, 'pipeline_xgboost',
    inputs)

## Storing the pipeline in a file

In [ ]:
MODEL_DIR = "./creditpredict"
MODEL_FILE_NAME = "credit_xgboost.onnx"

In [ ]:
os.mkdir(MODEL_DIR)
# And save.
with open(MODEL_DIR + "/" + MODEL_FILE_NAME, "wb") as f:
    f.write(model_onnx.SerializeToString())

In [ ]:
!ls -al

total 24
drwxr-xr-x 1 root root 4096 Dec  6 19:48 .
drwxr-xr-x 1 root root 4096 Dec  6 18:26 ..
drwxr-xr-x 4 root root 4096 Dec  3 14:33 .config
drwxr-xr-x 2 root root 4096 Dec  6 19:48 creditpredict
drwxr-xr-x 1 root root 4096 Dec  3 14:33 sample_data
drwxr-xr-x 7 root root 4096 Dec  6 18:56 wandb


### Uploading the pipeline to weights and biases for tracking

https://docs.wandb.ai/guides/artifacts/model-versioning#version-and-store-reliably

In [ ]:
os.environ["WANDB_API_KEY"] = "f38031d4fe33eb56cb0d138fc5da4115578dbf43"

In [ ]:
wandb.init(project='creditpredict', 
           config=params, 
           tags = ['XGB', 'OneHotEncoding'])

wandb.run.name = "XGBProdModel"

In [ ]:
#wandb.log( {"accuracy" : accuracy})

In [ ]:
model_artifact = wandb.Artifact("XGBoost_credit",
                                type = 'model',
                                description = 'XGBoost Model for credit class prediction')

In [ ]:
model_artifact.add_dir(MODEL_DIR)

wandb: Adding directory to artifact (./creditpredict)... Done. 0.1s


In [ ]:
wandb.run.log_artifact(model_artifact)

In [ ]:
wandb.save()
wandb.finish()

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


## Storing in Pickel format

In [ ]:
class CreditPredictionModel():
    
    def __init__(self, pipeline, all_features, cat_features, num_features, accuracy):
        self.pipeline = pipeline
        self.all_features = all_features
        self.cat_features = cat_features
        self.num_features = num_features
        self.cutoff = cutoff

In [ ]:
credit_model = CreditPredictionModel(reg, x_features, cat_features, num_features, accuracy)

In [ ]:
from joblib import dump, load
dump(credit_model, "creditmodel.pkl")

['creditmodel.pkl']